
# Smart Grid Mini System — Lab Robotika (2 UKM)
Notebook ini mencontohkan penerapan **Smart Grid: Fundamentals of Design and Analysis (James Momoh)** pada **Sistem Lab Robotika Kampus** dengan dua UKM robotik:
- **UKM A (Kompetisi)**: beban lebih tinggi & fluktuatif (latihan robot, printer 3D).
- **UKM B (Penelitian)**: beban lebih stabil (komputer, sensor, PLC).

## Fitur Notebook
1. **Simulasi Data Energi** (PV surya 2 kWp, baterai 10 kWh, batas daya grid).
2. **Kontrol Manajemen Energi**: prioritas sumber (PV → Baterai → Grid), pembatasan daya (*demand response*).
3. **Analisis & Visualisasi**: konsumsi, suplai, SOC baterai, kejadian *load shedding*.
4. **Peramalan Beban Sederhana** (regresi linear manual dgn NumPy).
5. **Penjadwalan Greedy** untuk aktivitas daya tinggi antar UKM agar tidak bentrok & hemat grid.

> Catatan: Semua komponen dibuat **tanpa ketergantungan ML eksternal** agar mudah dipakai di Colab maupun lokal.


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)
pd.options.display.float_format = '{:,.2f}'.format

# util plotting (sesuai aturan: 1 chart per plot, tidak setting warna khusus)
def _lineplot(df, x, y, title, xlabel, ylabel):
    plt.figure(figsize=(10,4))
    plt.plot(df[x], df[y])
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

def _multilineplot(df, x, y_cols, title, xlabel, ylabel):
    plt.figure(figsize=(10,4))
    for c in y_cols:
        plt.plot(df[x], df[c], label=c)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(True)
    plt.show()



## Konfigurasi Sistem
Parameter yang disederhanakan untuk mini-grid lab:
- PV: **2 kWp**, efisiensi profil harian sinus sederhana.
- Baterai: **10 kWh**, efisiensi coulombic 95%, **Pmax charge/discharge = 3 kW**.
- Grid (batas MCB lab): **5 kW**.
- Interval simulasi: **1 jam**, durasi **14 hari** (336 jam).
- Prioritas sumber: **PV → Baterai → Grid**.
- *Demand response*: saat total beban > ambang, beban non-kritis diputus mulai dari UKM prioritas lebih rendah.


In [ ]:

# ===== Konfigurasi =====
SIM_DAYS = 14
DT_H = 1  # 1 jam
N = SIM_DAYS * 24

PV_KWP = 2.0
BAT_KWH = 10.0
BAT_SOC_INIT = 0.6   # 60%
BAT_EFF = 0.95       # round-trip simplification per leg
P_BAT_MAX = 3.0      # kW charge/discharge limit

GRID_PMAX = 5.0      # kW batas daya grid (MCB)
DR_MARGIN = 0.9      # aktifkan DR jika prediksi konsumsi > 90% dari batas sumber aktif

# Profil beban dasar (kW)
BASE_A = 0.7  # UKM A (Kompetisi)
BASE_B = 0.5  # UKM B (Penelitian)

# Puncak waktu belajar/latihan (jam)
PEAK_A_HOURS = [9,10,11,14,15,16,20,21]  # lebih banyak sesi
PEAK_B_HOURS = [10,11,13,19]             # stabil

# Kenaikan beban saat peak (kW)
PEAK_A_ADD = 0.8
PEAK_B_ADD = 0.5

# Hari event kompetisi (meningkatkan A)
EVENT_DAYS = [5,6,12]  # index hari (0-based) dari 14 hari
EVENT_A_ADD = 0.8      # tambahan kW sepanjang hari event



## Simulasi Profil PV & Beban
- PV dimodelkan sebagai kurva sinus harian dengan puncak siang hari.
- Beban UKM memiliki pola jam puncak + noise.
- Hari **event** menaikkan profil UKM A.


In [ ]:

hours = np.arange(N)
day_index = hours // 24
hour_of_day = hours % 24

# PV profile: 0 malam, puncak siang (~12-13)
def pv_profile(hour):
    h = hour % 24
    if 6 <= h <= 18:
        # skala sinus dari 0 di 6/18 ke puncak di 12
        x = (h - 6) / 12.0 * np.pi
        return max(0.0, np.sin(x)) * PV_KWP  # kW instantaneous
    return 0.0

pv_kw = np.array([pv_profile(h) for h in hours])

# Beban UKM A & B (kW)
A_kw = np.full(N, BASE_A)
B_kw = np.full(N, BASE_B)

for i, h in enumerate(hour_of_day):
    if h in PEAK_A_HOURS:
        A_kw[i] += PEAK_A_ADD
    if h in PEAK_B_HOURS:
        B_kw[i] += PEAK_B_ADD

# Event boost utk A
for d in EVENT_DAYS:
    idx = (day_index == d)
    A_kw[idx] += EVENT_A_ADD

# Tambahkan noise kecil
A_kw += np.random.normal(0, 0.05, N)
B_kw += np.random.normal(0, 0.03, N)
A_kw = np.clip(A_kw, 0, None)
B_kw = np.clip(B_kw, 0, None)

df = pd.DataFrame({
    't': hours,
    'day': day_index,
    'hod': hour_of_day,
    'PV_kW': pv_kw,
    'Load_UKM_A_kW': A_kw,
    'Load_UKM_B_kW': B_kw,
})
df.head(8)



## Manajemen Energi & Demand Response
Algoritma per jam:
1. Penuhi beban dengan **PV** terlebih dahulu.
2. Kelebihan PV **mengisi baterai** (≤ *P_BAT_MAX*).
3. Kekurangan daya dicoba **dari baterai** (≤ *P_BAT_MAX*).
4. Sisa kekurangan diambil dari **grid** (≤ *GRID_PMAX*).
5. Jika **keterbatasan sumber** menyebabkan defisit, aktifkan **demand response (DR)**:
   - Lepaskan beban non-kritis mulai dari UKM prioritas **lebih rendah** (misal B dulu, lalu A).
   - Simulasi: 30% dari setiap UKM dianggap non-kritis dan dapat diputus.


In [ ]:

soc = np.zeros(N)          # 0..1
soc[0] = BAT_SOC_INIT
from_pv = np.zeros(N)      # kW dipakai beban
to_batt = np.zeros(N)      # kW charge
from_batt = np.zeros(N)    # kW discharge (ke beban)
from_grid = np.zeros(N)    # kW dari grid
shed_A = np.zeros(N)       # kW shed (A)
shed_B = np.zeros(N)       # kW shed (B)

NONCRIT_RATIO = 0.3        # bagian beban yang bisa di-shed

def charge_batt(energy_kwh, soc):
    cap_kwh = BAT_KWH
    e_free = cap_kwh * (1 - soc)
    # efisiensi pengisian
    e_stored = min(energy_kwh * BAT_EFF, e_free)
    soc_new = soc + e_stored / cap_kwh
    return soc_new, e_stored

def discharge_batt(need_kwh, soc):
    cap_kwh = BAT_KWH
    e_avail = soc * cap_kwh
    # efisiensi pengosongan (keluar)
    e_deliver = min(need_kwh, e_avail * BAT_EFF)
    soc_new = soc - (e_deliver / BAT_EFF) / cap_kwh
    return soc_new, e_deliver

for i in range(N):
    loadA = df.loc[i, 'Load_UKM_A_kW']
    loadB = df.loc[i, 'Load_UKM_B_kW']
    load_total = max(0.0, loadA) + max(0.0, loadB)

    # suplai PV dulu
    pv = df.loc[i, 'PV_kW']
    pv_to_load = min(pv, load_total)
    from_pv[i] = pv_to_load
    rem_load = load_total - pv_to_load
    pv_surplus = pv - pv_to_load

    # charge battery dari surplus PV (batas daya)
    charge_power = min(pv_surplus, P_BAT_MAX)
    if charge_power > 0:
        # dalam 1 jam, energy = power * 1h
        soc[i], charged = charge_batt(charge_power * 1.0, soc[i-1] if i>0 else soc[0])
        to_batt[i] = charged / 1.0  # kW-equivalent
    else:
        soc[i] = soc[i-1] if i>0 else soc[0]

    # jika masih ada beban sisa -> discharge battery (batas daya)
    if rem_load > 0:
        discharge_power = min(P_BAT_MAX, rem_load)
        soc[i], delivered = discharge_batt(discharge_power * 1.0, soc[i])
        from_batt[i] = delivered / 1.0
        rem_load -= from_batt[i]

    # sisa dari grid (batas daya GRID_PMAX)
    grid_supply = min(GRID_PMAX, rem_load)
    from_grid[i] = grid_supply
    rem_load -= grid_supply

    # Jika masih defisit -> Demand Response
    if rem_load > 1e-6:
        # target shed mulai dari UKM B (prioritas lebih rendah)
        # jumlah yang bisa di-shed per UKM = NONCRIT_RATIO * load UKM tsb
        shed_needed = rem_load

        # shed B dulu
        cap_B = NONCRIT_RATIO * loadB
        sB = min(cap_B, shed_needed)
        shed_B[i] = sB
        shed_needed -= sB

        # shed A kalau masih perlu
        if shed_needed > 1e-6:
            cap_A = NONCRIT_RATIO * loadA
            sA = min(cap_A, shed_needed)
            shed_A[i] = sA
            shed_needed -= sA

        # setelah shedding, anggap terpenuhi (karena beban dikurangi)
        rem_load = 0.0

# gabungkan hasil ke DataFrame
df['from_PV_kW'] = from_pv
df['to_Batt_kW'] = to_batt
df['from_Batt_kW'] = from_batt
df['from_Grid_kW'] = from_grid
df['SOC'] = soc
df['Shed_A_kW'] = shed_A
df['Shed_B_kW'] = shed_B

df.head(10)



## Visualisasi


In [ ]:

# PV vs Beban Total
df['Load_Total_kW'] = df['Load_UKM_A_kW'] + df['Load_UKM_B_kW']
_multilineplot(df, 't', ['PV_kW', 'Load_Total_kW'], 'PV vs Total Load', 'Jam', 'kW')


In [ ]:

# Alokasi Sumber Daya
_multilineplot(df, 't', ['from_PV_kW','from_Batt_kW','from_Grid_kW'], 'Suplai Daya per Sumber', 'Jam', 'kW')


In [ ]:

# State of Charge Baterai
_lineplot(df, 't', 'SOC', 'State of Charge (SOC) Baterai', 'Jam', 'SOC (0..1)')


In [ ]:

# Load Shedding
_multilineplot(df, 't', ['Shed_A_kW','Shed_B_kW'], 'Demand Response (Load Shedding)', 'Jam', 'kW shed')



## Peramalan Beban Sederhana (Regresi Linear Manual)
Kita memprediksi **beban per jam** untuk **hari ke-15** (besok) berdasarkan 14 hari historis.
Fitur:
- Satu-hot **jam-ke** (0..23)
- Satu-hot **hari-ke** (0..6) modulo hari (pola mingguan sederhana)

Model: **Normal equation** \( w = (X^T X)^{-1} X^T y \).


In [ ]:

def build_features(day_idx, hod):
    # one-hot jam 0..23
    x_hod = np.eye(24)[hod]
    # one-hot day of week 0..6 (anggap day_idx % 7)
    x_dow = np.eye(7)[day_idx % 7]
    # bias
    return np.concatenate([x_hod, x_dow, [1.0]])

def fit_linear(X, y):
    # normal equation with small ridge to avoid singularity
    XtX = X.T @ X
    ridge = 1e-6 * np.eye(XtX.shape[0])
    w = np.linalg.inv(XtX + ridge) @ X.T @ y
    return w

def predict_linear(X, w):
    return X @ w

# siapkan dataset untuk masing-masing UKM
def prepare_xy(load_series):
    X_list, y_list = [], []
    for i in range(len(load_series)):
        x = build_features(int(df.loc[i,'day']), int(df.loc[i,'hod']))
        X_list.append(x)
        y_list.append(load_series[i])
    X = np.vstack(X_list)
    y = np.array(y_list)
    return X, y

X_A, y_A = prepare_xy(df['Load_UKM_A_kW'].values)
X_B, y_B = prepare_xy(df['Load_UKM_B_kW'].values)

w_A = fit_linear(X_A, y_A)
w_B = fit_linear(X_B, y_B)

# prediksi untuk hari ke-14 (0-based -> hari ke-15 kalender)
future_day = 14
X_future_list = []
for h in range(24):
    X_future_list.append(build_features(future_day, h))
X_future = np.vstack(X_future_list)

pred_A = predict_linear(X_future, w_A)
pred_B = predict_linear(X_future, w_B)

pred_df = pd.DataFrame({
    'hour': np.arange(24),
    'pred_UKM_A_kW': pred_A,
    'pred_UKM_B_kW': pred_B,
    'pred_total_kW': pred_A + pred_B
})
pred_df.head()


In [ ]:

# Plot prediksi harian next day
_multilineplot(pred_df, 'hour', ['pred_UKM_A_kW','pred_UKM_B_kW','pred_total_kW'], 
               'Prediksi Beban per Jam (Hari+1)', 'Jam', 'kW')



## Penjadwalan Greedy Aktivitas Daya-Tinggi
Misal tiap UKM ingin mem-booking **aktivitas daya tinggi** berdurasi 3 jam dalam 1 hari berikutnya.
Tujuan: menjadwalkan slot supaya **tumpang tindih minimum** dan **mendekati jam PV tinggi**.

Strategi greedy:
1. Hitung **skor** untuk setiap jam: `skor = PV_prediksi - total_load_prediksi` → semakin besar semakin baik.
2. Jadwalkan UKM dengan prioritas **yang beban puncaknya lebih tinggi** terlebih dahulu (UKM A), pilih 3 jam berurutan dengan skor total terbaik tanpa bentrok.
3. Jadwalkan UKM B berikutnya dengan aturan yang sama, menghindari bentrok.


In [ ]:

# Estimasi PV next-day sederhana: gunakan profil PV rata-rata dari 14 hari
pv_by_hod = df.groupby('hod')['PV_kW'].mean().reindex(range(24)).values

score = pv_by_hod - pred_df['pred_total_kW'].values
score = score.astype(float)

def best_3h_block(occupied):
    best_sum = -1e9
    best_start = None
    for start in range(0, 22):  # 0..21 inclusive start for 3-hour block
        block = [start, start+1, start+2]
        if any(occupied[b] for b in block):
            continue
        s = score[start] + score[start+1] + score[start+2]
        if s > best_sum:
            best_sum = s
            best_start = start
    return best_start, best_sum

occupied = [False]*24
sched = {}

# Jadwalkan UKM A dulu
start_A, _ = best_3h_block(occupied)
for h in [start_A, start_A+1, start_A+2]:
    occupied[h] = True
sched['UKM_A'] = (start_A, start_A+2)

# Jadwalkan UKM B
start_B, _ = best_3h_block(occupied)
for h in [start_B, start_B+1, start_B+2]:
    occupied[h] = True
sched['UKM_B'] = (start_B, start_B+2)

sched


In [ ]:

# Visualisasi jadwal (garis waktu 24 jam)
timeline = ['']*24
for h in range(24):
    if sched['UKM_A'][0] <= h <= sched['UKM_A'][1]:
        timeline[h] = 'A'
    if sched['UKM_B'][0] <= h <= sched['UKM_B'][1]:
        timeline[h] = timeline[h] + 'B' if timeline[h] else 'B'

print("Jadwal 24 jam (A=UKM A, B=UKM B, AB=overlap):")
print(' '.join([f"{h:02d}:{timeline[h] or '-'}" for h in range(24)]))



## Ekspor Data
Simpan hasil simulasi ke CSV untuk dipakai di dashboard eksternal (Grafana/Node-RED/Excel).


In [ ]:

sim_csv_path = '/mnt/data/sim_smartgrid_lab_ukm.csv'
pred_csv_path = '/mnt/data/pred_nextday_lab_ukm.csv'

df.to_csv(sim_csv_path, index=False)
pred_df.to_csv(pred_csv_path, index=False)

sim_csv_path, pred_csv_path



## Ringkasan & Saran Eksperimen
- Anda dapat mengganti `GRID_PMAX`, `P_BAT_MAX`, atau `EVENT_DAYS` untuk melihat efeknya terhadap **SOC** dan **load shedding**.
- Tambahkan **prioritas granular** (mis. printer 3D non-kritis, PLC kritis) dengan memecah beban per-UKM menjadi komponen.
- Integrasikan dengan **MQTT** (ESP32) dan simpan data ke **InfluxDB** untuk dashboard real-time.
- Validasi model peramalan dengan memisah **train/test** dan menghitung **RMSE** per jam.
